#Kisore Senthilkumar 50610194

PROBLEM STATEMENT: ANALYSIS OF GENRES IMPACT ON THE RATING OF THE GAME BEING POSITIVE OR NOT

In [18]:
from ast import literal_eval
import itertools
import time
import re

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from warnings import simplefilter

pd.options.display.max_columns=100

#Data Preprocessing before it is used for the modelling

Firstly we identify the feature we need and make columns that would assist in the modelling.

Genres column could be separated into multiple columns that correspond one of the major values of genres and having binary values associated to indicate the presence of it in the game

The target value is calculated by  using the positive and negative rating columns then converted to binary.

Converting to binary aids in making it usable by the models off the bat.

We then make a sub dataset by only taking in the necessary columns and making it a completely binary dataset.

we then export it as a standalone csv file to be used by the models

In [9]:
df = pd.read_csv('steam.csv', engine='python', encoding='utf-8')
df.head(5)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [10]:
df['rnum']=df['positive_ratings']/(df['positive_ratings']+df['negative_ratings'])
df['rval'] = np.where(df['rnum'] > 0.5, 1, 0)
df

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,rnum,rval
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,0.973888,1
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,0.839787,1
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,0.895648,1
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,0.826623,1
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,0.947996,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09,1.000000,1
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69,0.888889,1
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99,0.000000,0
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19,1.000000,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27074 non-null  object 
 5   publisher         27061 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

In [12]:
def if_indie(genre):
    if 'Indie' in genre:
        return 1
    else:
        return 0
df['is_indie'] = df['genres'].apply(if_indie)

def if_action(genre):
    if 'Action' in genre:
        return 1
    else:
        return 0
df['is_action'] = df['genres'].apply(if_action)

def if_casual(genre):
    if 'Casual' in genre:
        return 1
    else:
        return 0
df['is_casual'] = df['genres'].apply(if_casual)

def if_strat(genre):
    if 'Strategy' in genre:
        return 1
    else:
        return 0
df['is_strat'] = df['genres'].apply(if_strat)

def if_rpg(genre):
    if 'RPG' in genre:
        return 1
    else:
        return 0
df['is_rpg'] = df['genres'].apply(if_rpg)

def if_adv(genre):
    if 'Adventure' in genre:
        return 1
    else:
        return 0
df['is_adv'] = df['genres'].apply(if_adv)

def if_f2p(genre):
    if 'Free to Play' in genre:
        return 1
    else:
        return 0
df['is_f2p'] = df['genres'].apply(if_f2p)

def if_sim(genre):
    if 'Simulation' in genre:
        return 1
    else:
        return 0
df['is_sim'] = df['genres'].apply(if_sim)

def if_ra(genre):
    if 'Racing' in genre:
        return 1
    else:
        return 0
df['is_ra'] = df['genres'].apply(if_ra)

def if_sports(genre):
    if 'Sports' in genre:
        return 1
    else:
        return 0
df['is_sports'] = df['genres'].apply(if_sports)

def if_mmo(genre):
    if 'Massively Multiplayer' in genre:
        return 1
    else:
        return 0
df['is_mmo'] = df['genres'].apply(if_mmo)


In [19]:
#Action, Adventure, Indie, Casual, Racing, Simulation, Free to Play, Massively Multiplayer, Strategy, Sports, RPG
ds=df[['is_action','is_indie','is_casual','is_strat','is_rpg','is_adv','is_f2p','is_sim','is_ra','is_sports','is_mmo','rnum','rval']]
ds.to_csv('drive/MyDrive/cleaned_data.csv')